
# Executive Dashboard Storyboard

This notebook assembles executive-ready visuals for the telecom subscriber dataset, focusing on the levers most strongly tied to churn and revenue. It is a lightweight storytelling layer that can be ported into a Streamlit or PowerPoint artifact.



## 1. Load data and configure plotting


In [1]:

import os
import sys
from pathlib import Path

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

NOTEBOOK_CWD = Path(os.getcwd()).resolve()
PROJECT_ROOT = None
for candidate in [
    NOTEBOOK_CWD,
    NOTEBOOK_CWD.parent,
    NOTEBOOK_CWD.parent.parent,
    NOTEBOOK_CWD / 'data_science_project'
]:
    src_dir = (candidate / 'src').resolve()
    if src_dir.exists():
        PROJECT_ROOT = candidate.resolve()
        if str(PROJECT_ROOT) not in sys.path:
            sys.path.append(str(PROJECT_ROOT))
        break

if PROJECT_ROOT is None:
    raise RuntimeError('Unable to locate project root for imports.')

DATA_PATH = PROJECT_ROOT / 'data' / 'processed' / 'clean_dataset.csv'
REPORT_FIG_DIR = PROJECT_ROOT / 'reports' / 'figures'
REPORT_FIG_DIR.mkdir(parents=True, exist_ok=True)

clean_df = pd.read_csv(DATA_PATH, parse_dates=['signup_date', 'last_seen'])
clean_df.head()


,customer_id,signup_date,last_seen,age,gender,province,lat,lng,plan_type,contract,...,defaulted_loan,next_month_spend,review_text,tenure_years,support_tickets_per_month,avg_monthly_revenue,spend_to_income_ratio,charges_per_gb,engagement_intensity,lifetime_value_projection
0,1,2020-08-15,2022-12-16,30,Male,Matabeleland North,-18.6203,27.6337,Prepaid,Month-to-Month,...,False,32.03,Fantastic experience from start to finish.,2.33,0.000,19.890000,0.027611,4.845266,42.33,941.28
1,2,2024-08-27,2025-05-11,31,Male,Mashonaland West,-17.2211,30.1817,Postpaid,One Year,...,False,43.09,"Excellent! Fast, reliable, and great support.",0.67,0.167,42.280000,0.088129,4.814234,16.74,855.32
2,3,2023-02-14,2023-03-30,38,Male,Manicaland,-19.0543,32.5927,Postpaid,Month-to-Month,...,False,26.88,"Excellent! Fast, reliable, and great support.",0.08,0.167,33.140000,0.028691,2.143939,29.46,33.14
3,4,2022-03-11,2022-12-04,57,Female,Masvingo,-20.5122,30.8098,Prepaid,One Year,...,False,20.44,Fantastic experience from start to finish.,0.67,0.167,19.198750,0.039658,1.307536,38.40,153.59
4,5,2019-02-03,2020-12-05,18,Male,Bulawayo,-20.4335,28.6515,Prepaid,Month-to-Month,...,True,12.45,Pretty satisfied with the features for the price.,1.83,0.000,14.855455,0.029844,3.683951,25.90,476.22



### KPI snapshot



## KPI definitions

| KPI | Definition | Why it matters |
| --- | --- | --- |
| Active customers | Count of current subscriber records | Sizing the opportunity and denominators for rate metrics |
| Churn rate | % of customers flagged churned = 1 | Primary retention KPI |
| High-support share | % with support tickets/month ? 0.5 | Quantifies pain point load |
| App adoption | % with has_app = True | Proxy for digital engagement |
| Average monthly revenue | Mean of vg_monthly_revenue | Revenue baseline |
| Next-month spend forecast | Mean of 
ext_month_spend | Forward-looking monetization |
| Premium cluster mix | % of customers in Cluster 0 | Value concentration |
| High-risk propensity share | % above churn score 75th percentile | Size of watchlist |
| Support-driven churn rate | Churn rate among support_segment = 0.5-1.5 | Measuring service friction |
| App-less churn lift | Churn rate difference between app and non-app users | Validating app as retention lever |


In [2]:

summary_metrics = {
    'Customers': f"{len(clean_df):,}",
    'Overall churn rate': f"{clean_df['churned'].mean()*100:.1f}%",
    'App adoption': f"{clean_df['has_app'].mean()*100:.1f}%",
    'Avg next-month spend': f"$ {clean_df['next_month_spend'].mean():.2f}",
    'Avg support tickets/mo': f"{clean_df['support_tickets_per_month'].mean():.2f}"
}
display_df = pd.DataFrame(summary_metrics, index=['Value']).T
fig = go.Figure(data=[go.Table(
    header=dict(values=['Metric', 'Value'], fill_color='#0d6efd', font=dict(color='white'), align='left'),
    cells=dict(values=[display_df.index, display_df['Value']], fill_color='#f8f9fa', align='left')
)])
fig.update_layout(title='Executive KPI Highlights')
fig.write_html(REPORT_FIG_DIR / 'dashboard_kpi_table.html')
fig



## 2. Retention pressure points



### 2.1 Churn by plan tier


In [3]:

churn_plan = clean_df.groupby('plan_type')['churned'].mean().reset_index()
churn_plan['churn_rate'] = churn_plan['churned'] * 100
fig = px.bar(
    churn_plan.sort_values('churn_rate', ascending=False),
    x='plan_type',
    y='churn_rate',
    color='plan_type',
    text='churn_rate',
    labels={'plan_type': 'Plan Type', 'churn_rate': 'Churn Rate (%)'},
    title='Premium customers churn at the highest rate despite higher value'
)
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(showlegend=False, yaxis_tickformat='.0f')
fig.write_html(REPORT_FIG_DIR / 'dashboard_churn_by_plan.html')
fig



### 2.2 Mobile app adoption


In [4]:

app_churn = (
    clean_df.assign(app_segment=clean_df['has_app'].map({True: 'Has mobile app', False: 'No mobile app'}))
    .groupby('app_segment')['churned']
    .mean()
    .reset_index()
)
app_churn['churn_rate'] = app_churn['churned'] * 100
fig = px.bar(
    app_churn.sort_values('churn_rate', ascending=False),
    x='app_segment',
    y='churn_rate',
    color='app_segment',
    text='churn_rate',
    labels={'app_segment': 'App Adoption', 'churn_rate': 'Churn Rate (%)'},
    title='Mobile app users churn 10 points less than non-users'
)
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(showlegend=False)
fig.write_html(REPORT_FIG_DIR / 'dashboard_churn_by_app.html')
fig



### 2.3 Service friction


In [5]:

support_bins = pd.cut(
    clean_df['support_tickets_per_month'],
    bins=[0, 0.2, 0.5, 1.5],
    labels=['0 - 0.2', '0.2 - 0.5', '0.5 - 1.5'],
    include_lowest=True
)
support_churn = (
    clean_df.assign(support_segment=support_bins)
    .groupby('support_segment')['churned']
    .mean()
    .reset_index()
)
support_churn['churn_rate'] = support_churn['churned'] * 100
fig = px.bar(
    support_churn,
    x='support_segment',
    y='churn_rate',
    color='support_segment',
    text='churn_rate',
    labels={'support_segment': 'Support tickets per month', 'churn_rate': 'Churn Rate (%)'},
    title='Heavy support users churn at outsized rates'
)
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(showlegend=False)
fig.write_html(REPORT_FIG_DIR / 'dashboard_churn_by_support.html')
fig


C:\Users\kkadema.SIRDC\AppData\Local\Temp\ipykernel_23736\4280195897.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.




## 3. Geographic hotspots


In [6]:

province_summary = (
    clean_df.groupby('province')
    .agg(customers=('customer_id', 'count'), churn_rate=('churned', 'mean'), avg_spend=('next_month_spend', 'mean'))
    .reset_index()
)
province_summary['churn_pct'] = province_summary['churn_rate'] * 100
fig = px.scatter_geo(
    province_summary,
    lat=clean_df.groupby('province')['lat'].mean().values,
    lon=clean_df.groupby('province')['lng'].mean().values,
    color='churn_pct',
    size='customers',
    hover_name='province',
    hover_data={'churn_pct': ':.1f', 'customers': ':,'},
    color_continuous_scale='Viridis',
    title='Churn concentration by province',
    projection='natural earth'
)
fig.update_layout(height=500)
fig.write_html(REPORT_FIG_DIR / 'dashboard_geo_churn.html')
fig



### Interactive controls

Use the dropdowns/sliders below to inspect churn and spend trajectories for specific plan types or provinces. Adjusting the filters will update the trend chart in real time.


In [7]:
!pip install ipywidgets


## 4. Revenue pulse


In [ ]:

import ipywidgets as widgets

plan_dropdown = widgets.Dropdown(
    options=['All'] + sorted(clean_df['plan_type'].unique()),
    value='All',
    description='Plan:'
)
province_dropdown = widgets.Dropdown(
    options=['All'] + sorted(clean_df['province'].unique()),
    value='All',
    description='Province:'
)

@widgets.interact(plan=plan_dropdown, province=province_dropdown)
def update_trend(plan='All', province='All'):
    data = clean_df.copy()
    if plan != 'All':
        data = data[data['plan_type'] == plan]
    if province != 'All':
        data = data[data['province'] == province]
    metrics = (
        data.assign(month=data['last_seen'].dt.to_period('M'))
        .groupby('month')
        .agg(churn_rate=('churned', 'mean'), next_spend=('next_month_spend', 'mean'))
        .reset_index()
    )
    metrics['month'] = metrics['month'].dt.to_timestamp()
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=metrics['month'],
        y=metrics['churn_rate'] * 100,
        name='Churn rate (%)',
        marker_color='#d63384',
        opacity=0.6
    ))
    fig.add_trace(go.Scatter(
        x=metrics['month'],
        y=metrics['next_spend'],
        name='Next month spend (USD)',
        line=dict(color='#0d6efd', width=3),
        yaxis='y2'
    ))
    fig.update_layout(
        title=f'Churn & spend trend ? {plan} | {province}',
        xaxis=dict(title='Month'),
        yaxis=dict(title='Churn rate (%)', rangemode='tozero'),
        yaxis2=dict(title='Next month spend (USD)', overlaying='y', side='right'),
        legend=dict(orientation='h', y=1.1, x=0.1)
    )
    return fig.show()


interactive(children=(Dropdown(description='Plan:', options=('All', 'Postpaid', 'Premium', 'Prepaid'), value='…

In [9]:

monthly_metrics = (
    clean_df.assign(month=clean_df['last_seen'].dt.to_period('M'))
    .groupby('month')
    .agg(churn_rate=('churned', 'mean'), next_spend=('next_month_spend', 'mean'))
    .reset_index()
)
monthly_metrics['month'] = monthly_metrics['month'].dt.to_timestamp()
fig = go.Figure()
fig.add_trace(go.Bar(
    x=monthly_metrics['month'],
    y=monthly_metrics['churn_rate'] * 100,
    name='Churn rate (%)',
    marker_color='#d63384',
    opacity=0.6,
    yaxis='y1'
))
fig.add_trace(go.Scatter(
    x=monthly_metrics['month'],
    y=monthly_metrics['next_spend'],
    name='Next month spend (USD)',
    line=dict(color='#0d6efd', width=3),
    yaxis='y2'
))
fig.update_layout(
    title='Churn spikes and spend dips move together',
    xaxis=dict(title='Month'),
    yaxis=dict(title='Churn rate (%)', rangemode='tozero'),
    yaxis2=dict(title='Next month spend (USD)', overlaying='y', side='right'),
    legend=dict(orientation='h', y=1.1, x=0.1)
)
fig.write_html(REPORT_FIG_DIR / 'dashboard_churn_spend_trend.html')
fig



## 5. Export snapshot


In [10]:

export_summary = {
    'kpi_table': REPORT_FIG_DIR / 'dashboard_kpi_table.html',
    'churn_by_plan': REPORT_FIG_DIR / 'dashboard_churn_by_plan.html',
    'churn_by_app': REPORT_FIG_DIR / 'dashboard_churn_by_app.html',
    'churn_by_support': REPORT_FIG_DIR / 'dashboard_churn_by_support.html',
    'geo_churn': REPORT_FIG_DIR / 'dashboard_geo_churn.html',
    'churn_spend_trend': REPORT_FIG_DIR / 'dashboard_churn_spend_trend.html'
}
pd.Series(export_summary)


kpi_table            C:\Users\kkadema.SIRDC\Desktop\data analysis\d...
churn_by_plan        C:\Users\kkadema.SIRDC\Desktop\data analysis\d...
churn_by_app         C:\Users\kkadema.SIRDC\Desktop\data analysis\d...
churn_by_support     C:\Users\kkadema.SIRDC\Desktop\data analysis\d...
geo_churn            C:\Users\kkadema.SIRDC\Desktop\data analysis\d...
churn_spend_trend    C:\Users\kkadema.SIRDC\Desktop\data analysis\d...
dtype: object